# Object-Oriented Basics

* **Classes & instances**

  - attributes, methods, and the role of `self`.
 
* **Construction & representation**

  - `__init__`, `__repr__`, `__str__`.
 
* **Equality & identity**

  - `==` vs `is`; custom `__eq__` basics.
 
* **Inheritance vs composition**

  - reuse an interface or assemble parts.
 
* **Properties**

  - computed attributes with validation; light encapsulation.

## Classes and Instances

In [1]:
class Position:
    def __init__(self, symbol: str, qty: int):
        self.symbol = symbol
        self.qty = qty
    def value(self, price: float) -> float:
        return self.qty * price
    def __repr__(self) -> str:
        return f"Position({self.symbol!r}, qty={self.qty})"

In [2]:
p = Position("AAPL", 100)

In [3]:
p, p.value(190.5)

(Position('AAPL', qty=100), 19050.0)

In [4]:
type(p).__name__, isinstance(p, Position)

('Position', True)

## Construction and Representation

In [5]:
class Trade:
    def __init__(self, symbol: str, qty: int, price: float):
        self.symbol = symbol
        self.qty = qty
        self.price = price
    def __repr__(self):
        return f"Trade({self.symbol!r}, qty={self.qty}, price={self.price})"
    def __str__(self):
        return f"{self.qty}x {self.symbol} @ {self.price}"

In [6]:
t = Trade("MSFT", 50, 420.0)

In [8]:
t  # repr for developers

Trade('MSFT', qty=50, price=420.0)

In [9]:
print(t)  # str for users

50x MSFT @ 420.0


## Equality and Identity

In [10]:
class Point:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    def __eq__(self, other):
        if isinstance(other, Point):
            return (self.x, self.y) == (other.x, other.y)
        return NotImplemented

In [11]:
a = Point(1, 2); b = Point(1, 2)

In [12]:
a == b, a is b

(True, False)

## Inheritance vs. Composition

In [14]:
class Equity(Position):
    def __init__(self, symbol: str, qty: int, currency: str = "USD"):
        super().__init__(symbol, qty)
        self.currency = currency

In [15]:
e = Equity("MSFT", 50) 

In [16]:
e.value(420.0)

21000.0

In [17]:
class Portfolio:
    def __init__(self):
        self.positions = []  # holds Position objects
    def add(self, pos: Position) -> None:
        self.positions.append(pos)
    def market_value(self, price_lookup: dict) -> float:
        return sum(p.value(price_lookup[p.symbol]) for p in self.positions)

In [18]:
prices = {"AAPL": 190.5, "MSFT": 420.0}

In [19]:
pf = Portfolio(); pf.add(Position("AAPL", 100)); pf.add(Equity("MSFT", 50))

In [20]:
pf.market_value(prices)

40050.0

## Properties and Light Encapsulation

In [21]:
class Account:
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self._balance = balance  # underscore: internal convention
    @property
    def balance(self) -> float:
        return self._balance
    @balance.setter
    def balance(self, value: float) -> None:
        if value < 0:
            raise ValueError("balance cannot be negative")
        self._balance = value

In [22]:
a = Account("Ada", 100.0); a.balance

100.0

In [23]:
a.balance = 125.0; a.balance

125.0

In [24]:
a.balance = -5

ValueError: balance cannot be negative

## Common Gotchas

* **Forgetting `self`**

  - instance methods must accept `self` first; calls pass it implicitly.
 
* **Shared mutable class attributes**

  - a list/dict defined on the class is shared across instances; place it on `self` in `__init__`.
 
* **Equality traps**

  - without `__eq__`, `==` falls back to identity; define it for value objects.
 
* **Overusing inheritance**

  - prefer composition unless your subtype truly "is a" parent type.
 
* **Name mangling confusion**

  - double underscores trigger name mangling; use a single underscore for "internal by convention."

## Exercises

**Business card class**

Create `Contact(name, email)` with `__repr__` and `__str__`. Add a method `domain()` returning the email domain.

In [25]:
class Contact:
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email
    def __repr__(self):
        return f"Contact({self.name!r}, email={self.email})"
    def __str__(self):
        return f"name={self.name}, email={self.email}"
    def domain(self):
        return self.email.split("@")[1]

In [26]:
contact = Contact("Boggs Dixon", "boggs@dixon.me")
contact

Contact('Boggs Dixon', email=boggs@dixon.me)

In [29]:
print(f"{contact.name} hosts email address at domain {contact.domain()}")

Boggs Dixon hosts email address at domain dixon.me


**Value object equality**

Implement `Money(amount, currency)` so two instances compare equal when fields match.

In [30]:
class Money:
    def __init__(self, amount: float, currency: str):
        self.amount = amount
        self.currency = currency
    def __repr__(self):
        return f"Money({self.amount}, currency={self.currency})"
    def __str__(self):
        return f"{self.currency} {self.amount}"
    def __eq__(self, other):
        if isinstance(other, Money):
            return (self.amount, self.currency) == (other.amount, other.currency)
        return ValueError(f"{other} is not of type Money")

In [33]:
sterling = "GBP"
cash1 = Money(50+50, sterling); cash2 = Money(80+20, sterling)

if cash1 == cash2:
    print(f"equal: {cash1}, {cash2}")

equal: GBP 100, GBP 100


**Portfolio composition**

Build a `Portfolio` that holds `Position`s and computes total value from a price map.

In [48]:
class Portfolio:
    def __init__(self):
        self.positions = []  # holds Position objects
    def add(self, pos: Position) -> None:
        self.positions.append(pos)
    def market_value(self, price_lookup: dict) -> float:
        return sum(p.value(price_lookup[p.symbol]) for p in self.positions)

In [49]:
syms = ["UTEN", "UTWO", "TLT", "SPY", "VTWAX"]
sizes = [30, 30, 30, 50, 50]
quotes = [44.08, 48.60, 87.40, 680.73, 50]

pf = Portfolio()
for i in range(len(syms)):
    pf.add(Position(syms[i], sizes[i]))
    prices[syms[i]] = quotes[i]

print(pf.market_value(prices))

41938.9


**Property validation**

Add a `price` property to `Trade` that forbids negatives.

In [44]:
class Trade:
    def __init__(self, symbol: str, qty: int, price: float):
        self.symbol = symbol
        self.qty = qty
        if price < 0:
            raise ValueError("price cannot be negative")
        self._price = price
    def __repr__(self):
        return f"Trade({self.symbol!r}, qty={self.qty}, price={self.price})"
    def __str__(self):
        return f"{self.qty}x {self.symbol} @ {self.price}"
    @property
    def price(self) -> float:
        return self._price
    @price.setter
    def price(self, price: float) -> None:
        if price < 0:
            raise ValueError("price cannot be negative")
        self._price = price

In [45]:
t = Trade("MSFT", 50, 420.0)

In [46]:
t.price = -200

ValueError: price cannot be negative

In [47]:
t2 = Trade("HPE", 10, -10)

ValueError: price cannot be negative

**Subclass specialization**

Subclass `Position` into `OptionPosition` with fields `kind` ("call"/"put") and `multiplier` (default 100). Override `value(price)` accordingly.

In [51]:
class OptionPosition(Position):
    def __init__(self, symbol: str, qty: int, kind: str, multiplier: int = 100):
        super().__init__(symbol, qty)
        if kind not in ("call", "put"):
            raise ValueError(f"invalid option kind: {kind}")
        self.kind = kind
        self.multiplier = multiplier
    def value(self, price: float) -> float:
        return self.qty * self.multiplier * price
    def __repr__(self) -> str:
        return f"OptionPosition({self.symbol!r}, qty={self.qty}, kind={self.kind}, multiplier={self.multiplier})"

In [52]:
o = OptionPosition("AAPL", 2, "strangle")

ValueError: invalid option kind: strangle

In [54]:
o = OptionPosition("AAPL", 2, "put")
o.value(220)

44000